In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name switch files to train second person
        
        file_name_path = './facetask/harry/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

# 𝐌𝐨𝐝𝐞𝐥 𝐓𝐫𝐚𝐢𝐧𝐢𝐧𝐠 𝐟𝐨𝐫 𝐭𝐰𝐨 𝐏𝐞𝐨𝐩𝐥𝐞.....

In [4]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './facetask/first/'
data_path1='./facetask/harry/'

onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
onlyfiles1 = [f1 for f1 in listdir(data_path1) if isfile(join(data_path1, f1))]

# Create arrays for training data and labels
Training_Data, Labels = [], []
Training_Data1, Labels1 = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
    
for j, files in enumerate(onlyfiles1):
    image_path1 = data_path1 + onlyfiles1[j]
    images1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    Training_Data1.append(np.asarray(images1, dtype=np.uint8))
    Labels1.append(j)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)
#creating a numpy array for second person data and labels
Labels1 = np.asarray(Labels1, dtype=np.int32)


priyanshu_model= cv2.face_LBPHFaceRecognizer.create()
# Let's train Priynshu  model 
priyanshu_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("*******Model  for Priyanshu trained sucessefully********")

harry_model= cv2.face_LBPHFaceRecognizer.create()
# Let's train our second  model 
harry_model.train(np.asarray(Training_Data1), np.asarray(Labels1))
print("******* Model trained for second person sucessefully*********")


*******Model  for Priyanshu trained sucessefully********
******* Model trained for second person sucessefully*********


# 𝐅𝐢𝐧𝐚𝐥 𝐌𝐨𝐯𝐞 𝐅𝐚𝐜𝐞 𝐑𝐞𝐜𝐨𝐠𝐧𝐢𝐭𝐢𝐨𝐧.....

In [6]:
import cv2
import numpy as np
import pywhatkit as py
import os
import pyttsx3
import time


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # "results" comprises of a tuple containing the label and the confidence value
        results = priyanshu_model.predict(face)
        results1=harry_model.predict(face)
        
        
        #SETTING UP CONFIDENCE SCORE FOR MY FACE
        if results[1]< 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
            
         #SETTING  UP CONFIDENCE SCORE FOR HARRY FACE   
        if results1[1]<500:
            confidence_harry= int( 100 * (1 - (results1[1])/400) )
            display_string1 = str(confidence_harry) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence> 85:
            cv2.putText(image, "Hey Priyanshu !!", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image)
            ctr=1
            #ctr 1 for my face it will send whatsapp and email
            break
            
            
        if confidence_harry>85:
            cv2.putText(image, "Hey purva !!", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image)
            ctr=2 
            #ctr 2 for harry face it will launch an instance and attach volume
            break
            
        else:
            
            cv2.putText(image, "I dont know, how r u ", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            engine=pyttsx3.init()
            engine.setProperty("rate",160)
            engine.say("Sorry , I know only Priyanshu and Harry Styles") 
            engine.runAndWait()
            #pyttsx3.speak(" I know ,Prriyanshu ,and Hary Styles ,only ,sorry")

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is theEnter Key
        break
cap.release()        
cv2.destroyAllWindows()



#email and whatsapp
if ctr==1:
    #subprocess.run('engine=pyttsx3.init() engine.setProperty("rate",160) engine.say("hello Priyanshu , sending WhatsApp and Email now") engine.runAndWait()',shell=True) 
    #pyttsx3.speak("Hello ,Priyanshu , Sending  WhatsApp and Email now")
    engine=pyttsx3.init()
    engine.setProperty("rate",160)
    engine.say("hello Priyanshu , sending WhatsApp and Email now") 
    engine.runAndWait()
    
    
    py.sendwhatmsg_instantly("+91 7302469023","Face detection")
    py.sendMail("exampleemail1822@gmail.com","example2345@##$%","exampleemail1822@gmail.com","FACE DETECTED")
    print("mail sent")
    
    
#FOR LAUNCHING INSTANCE AND ATTACHING STORAGE    

elif ctr==2:
    engine=pyttsx3.init()
    engine.setProperty("rate",160)
    engine.say("hello Harry Styles ,Launching ec2 instance and adding volume to it now") 
    engine.runAndWait()
    
    #Launching instance commands
    
    os.system("aws ec2 run-instances --image-id ami-0ad704c126371a549  --instance-type t2.micro  --count 1  --subnet-id subnet-7b98c237 --security-group-ids sg-0dae0a1d6032375b2 --key-name first_os_key >ec2.txt")
    print("Your Instance Had been Launched")
    #storage
    os.system(" aws ec2 create-volume --availability-zone ap-south-1b  --size 5  --volume-type gp2 >ebs.txt")
    print("volume created")
    time.sleep(30)
    engine=pyttsx3.init()
    engine.setProperty("rate",160)
    engine.say("To attach volume please  run the next Cell") 
    engine.runAndWait()

<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-6-2bec07cf3c62>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Your Instance Had been Launched
volume created


In [7]:
#Attaching 5GB volume to created instance

os.system("aws ec2 attach-volume --volume-id   vol-07f3bf3d518f2657d --instance-id i-04d3e9e33cf79af78    --device /dev/xvdi")
engine=pyttsx3.init()
engine.setProperty("rate",160)
engine.say("Thank you ,Your volume is attached now") 
engine.runAndWait()


In [ ]:
#SEPERATE TRAINING FOR MY FACE

'''import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './facetask/first/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)


priyanshu_model= cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
priyanshu_model.train(np.asarray(Training_Data), np.asarray(Labels))
print(" Priyanshu Model trained sucessefully")'''

In [ ]:
#SEPERATE TRAINING FOR SECOND PERSON(HARRY) FACE


'''import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './facetask/harry/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

harry_model= cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
harry_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("harry Model trained sucessefully")'''